In [1]:
using Pkg
Pkg.activate("/media/mat/HDD/AdaptiveTransportMap/")

 Activating environment at `/media/mat/HDD/AdaptiveTransportMap/Project.toml`


In [7]:
using Revise
using AdaptiveTransportMap
using LinearAlgebra
using Test
using ForwardDiff
using SpecialFunctions
using Polynomials
using BenchmarkTools
using TransportMap
using QuadGK
using FastGaussQuadrature
using AdaptiveTransportMap: derivative, vander, scale!
using DiffResults
using QuadGK
using Distributions

### Basis

In [18]:
    Nx = 1
    Ne = 100
    ens = EnsembleState(Nx, Ne)

    ens.S .= 1.0 .+ 5.0*randn(Nx, Ne)

    scale!(ens; diag = true)

    @test norm(mean(ens))<1e-10
    @test norm(cov(ens) .- 1.0)<1e-10

Test Passed

In [20]:
cov(ens)

1×1 Array{Float64,2}:
 0.9999999999999998

In [3]:
Nx = 6
B = CstProHermite(7)
MB = MultiBasis(B, Nx)

MultiBasis{9,6}(Basis of 9 functions:
ProPolyHermite{0}(ImmutablePolynomial(1.0), false)
ProHermite{0}(ProPolyHermite{0}(ImmutablePolynomial(1.0), false), false)
ProHermite{1}(ProPolyHermite{1}(ImmutablePolynomial(1.0*x), false), false)
ProHermite{2}(ProPolyHermite{2}(ImmutablePolynomial(-1.0 + 1.0*x^2), false), false)
ProHermite{3}(ProPolyHermite{3}(ImmutablePolynomial(-3.0*x + 1.0*x^3), false), false)
ProHermite{4}(ProPolyHermite{4}(ImmutablePolynomial(3.0 - 6.0*x^2 + 1.0*x^4), false), false)
ProHermite{5}(ProPolyHermite{5}(ImmutablePolynomial(15.0*x - 10.0*x^3 + 1.0*x^5), false), false)
ProHermite{6}(ProPolyHermite{6}(ImmutablePolynomial(-15.0 + 45.0*x^2 - 15.0*x^4 + 1.0*x^6), false), false)
ProHermite{7}(ProPolyHermite{7}(ImmutablePolynomial(-105.0*x + 105.0*x^3 - 21.0*x^5 + 1.0*x^7), false), false)
)

In [4]:
@btime ForwardDiff.gradient(x->B[5](x[1]),[5.0])

  1.867 μs (9 allocations: 480 bytes)


1-element Array{Float64,1}:
 -0.39188218965422494

In [5]:
@btime ForwardDiff.derivative(x->B[5](x),5.0)

  107.739 ns (6 allocations: 176 bytes)


-0.39188218965422494

In [6]:
function timing()
    f(x) = (x^5-4*x^3+1)*exp(-x^2/2)  
    @btime ForwardDiff.gradient(x->f(x[1]),[5.0])
    @btime ForwardDiff.derivative(x->f(x),5.0)
end

timing (generic function with 1 method)

In [51]:
FamilyPhyHermite[5].P

PhyHermite{4}(PhyPolyHermite{4}(ImmutablePolynomial(12.0 - 48.0*x^2 + 16.0*x^4), false), false)

In [8]:
@btime quadgk(x->FamilyPhyHermite[5](x), 0, 1.0)

  2.024 μs (137 allocations: 2.34 KiB)


(0.5630021473036511, 1.4497070210950369e-11)

In [9]:
Nx = 10
Nψ = 8
B = CstProHermite(4)
MB = MultiBasis(B, Nx)
f = ExpandedFunction([MultiFunction(MB, rand(1:6,10)) for i=1:5], randn(5));

In [10]:
result = DiffResults.HessianResult(randn(10));
@time result = ForwardDiff.gradient!(result, f, randn(10));

MethodError: MethodError: no method matching getindex(::MultiBasis{6,10}, ::Int64)

In [11]:
first()

MethodError: MethodError: no method matching first()
Closest candidates are:
  first(!Matched::Tuple{}) at tuple.jl:95
  first(!Matched::BitSet) at bitset.jl:356
  first(!Matched::Core.Compiler.StmtRange) at show.jl:1848
  ...

In [12]:
function timing2()
    @btime x = randn(100,20)
    @btime begin 
        x = randn(20,200)
        for i=1:200
        x[10,i]
        end
    end
end

timing2 (generic function with 1 method)

In [13]:
timing2()

  10.325 μs (1 allocation: 15.75 KiB)
  20.985 μs (2 allocations: 31.33 KiB)


In [14]:
function timing()
    Nx = 10
    Nψ = 8
    B = CstProHermite(4)
    MB = MultiBasis(B, Nx);
    f = ExpandedFunction([MultiFunction(MB, rand(1:6,10)) for i=1:5], randn(5));
    [f(randn(10)) for i=1:200]
end

timing (generic function with 1 method)

In [15]:
@btime timing()

MethodError: MethodError: no method matching getindex(::MultiBasis{6,10}, ::Int64)

In [16]:
@time [f(randn(10)) for i=1:200]

MethodError: MethodError: no method matching getindex(::MultiBasis{6,10}, ::Int64)

In [18]:
function timing() 
    f = MultiFunction(MB, rand(1:6,10));
    result = DiffResults.HessianResult(randn(10));
    result = ForwardDiff.hessian!(result, f, randn(10));
end

timing (generic function with 1 method)

In [19]:
function timing2() 
    f = MultiFunction(MB, rand(1:6,10));
    x = randn(10)
    f(x)
    ForwardDiff.gradient(f, x);
    ForwardDiff.hessian(f, x);
end

timing2 (generic function with 1 method)

### Hermite functions

In [68]:
FamilyScaledPhyHermite[5]

PhyHermite{4}(PhyPolyHermite{4}(ImmutablePolynomial(0.4599685791773267 - 1.8398743167093068*x^2 + 0.6132914389031022*x^4), true), true)

In [61]:
integral, err = quadgk(x-> (12.0 -48*x^2 + 16*x^4)*exp(-x^2/2), 0.0, 1.0, rtol=1e-12)

(0.5630021473036511, 8.881784197001252e-16)

In [103]:
function integrate(f,a,b)
    return quadgk(x-> f(x), a, b, rtol=1e-12)[1]
end

integrate (generic function with 1 method)

In [107]:
function timing()
    @btime integrate(FamilyScaledPhyHermite[7], 0, 2.0)
    @btime derivative(FamilyScaledPhyHermite[7], -1, [2.0])
end

timing (generic function with 1 method)

In [108]:
timing()

  10.976 μs (848 allocations: 13.86 KiB)
  3.005 μs (108 allocations: 3.39 KiB)


1-element Array{Float64,1}:
 -0.08868073032617163